## Import packages

In [12]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import os.path as osp
import os
import argparse

## Get home_path
We get `home_path` on different machines

In [5]:
home_path = osp.expanduser('~')
home_path

'/Users/quanguet'

## Compute the IoU
We define the function to compute the IoU of a predicted box with a list of ground truth boxes.

A box is represented by a list `[xmin, ymin, xmax, ymax]`.

A list ground truth boxes is a list of boxes `[box1, box2, box3]`.

All are `np.array`


In [7]:
def get_iou(pred_img_box, gt_img_boxes):
    # Compute the area of all boxes including the predicted box
    pred_box_area  = (pred_img_box[2] - pred_img_box[0] + 1) # area = w
    pred_box_area *= (pred_img_box[3] - pred_img_box[1] + 1) # area = w * h
    gt_boxes_area  = (gt_img_boxes[:, 2] - gt_img_boxes[:, 0] + 1)  # area = w
    gt_boxes_area *= (gt_img_boxes[:, 3] - gt_img_boxes[:, 1] + 1) # area = w * h

    xx1 = np.maximum(pred_img_box[0], gt_img_boxes[:, 0])
    yy1 = np.maximum(pred_img_box[1], gt_img_boxes[:, 1])
    xx2 = np.minimum(pred_img_box[2], gt_img_boxes[:, 2])
    yy2 = np.minimum(pred_img_box[3], gt_img_boxes[:, 3])

    # Compute the area of intersection
    w = np.maximum(0.0, xx2 - xx1 + 1)
    h = np.maximum(0.0, yy2 - yy1 + 1)

    inter = w * h
    # get IoU all boxes with the box of highest conf
    iou = inter / (pred_box_area + gt_boxes_area[:] - inter)
    return iou


## Check whether a predict box is a ground truth

In [8]:
def pred_box_is_gt_box(pred_img_box, gt_img_boxes, threshold=0.5):
    iou = get_iou(pred_img_box, gt_img_boxes)
    return np.any(iou > threshold)

## Get the prediction from a file

In [15]:
# args = argparse.ArgumentParser().parse_args()

result_path = 'Dropbox/DeepFPNResnet/Resnet101/afw_Deepresnet101_val.txt'
data_root = 'Datasets/AFW'

def get_pred_data():
    # file_name score 
    boxes = {}
    scores = {}
    img_names = []
    img_paths = []

    with open(osp.join(home_path, result_path), "r") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        elems = line.split(" ")
        img_name = elems[0] + '.jpg'
        score = float(elems[1])
        box = elems[2:]
        box = [float(value) for value in box]

        if len(img_names) == 0 or img_name != img_names[-1]:
            img_names.append(img_name)
            img_paths.append(osp.join(home_path, data_root, img_name))

        if boxes.get(img_name) is None:
            boxes[img_name] = []
            scores[img_name] = []
        boxes[img_name].append(box)
        scores[img_name].append(score)

    final_boxes, final_scores = [], []
    for img_name in img_names:
        final_boxes.append(np.array(boxes[img_name]))
        final_scores.append(np.array(scores[img_name]))

    return img_names, img_paths, final_boxes, final_scores

img_names, img_paths, final_boxes, final_scores = get_pred_data()

In [17]:
img_names[:5]

['2353849.jpg',
 '2201628776.jpg',
 '406798473.jpg',
 '70037463.jpg',
 '2030653815.jpg']

In [18]:
img_paths[:5]

['/Users/quanguet/Datasets/AFW/2353849.jpg',
 '/Users/quanguet/Datasets/AFW/2201628776.jpg',
 '/Users/quanguet/Datasets/AFW/406798473.jpg',
 '/Users/quanguet/Datasets/AFW/70037463.jpg',
 '/Users/quanguet/Datasets/AFW/2030653815.jpg']

In [19]:
final_boxes[:5]

[array([[ 277. ,  478. ,  471. ,  791.3],
        [ 971.3,  444.7, 1115.3,  631.7],
        [ 678.7,  894.2,  783.6, 1030.6]]),
 array([[576. , 338.7, 755.3, 590.7],
        [730.1, 103. , 742.1, 119. ],
        [719.2, 132.6, 739.8, 156.6]]),
 array([[517.7, 370.7, 649.3, 545.7],
        [312. , 346.3, 446. , 513. ],
        [818.3, 375.3, 966.3, 572. ]]),
 array([[ 470. ,  193.3,  681.3,  517.7],
        [1011. ,  406.7, 1259.3,  733.3]]),
 array([[724.3, 435. , 914. , 681. ],
        [529. , 459.8, 636.2, 659.2]])]

In [20]:
final_scores[:5]

[array([1.   , 1.   , 0.998]),
 array([1.   , 0.883, 0.637]),
 array([1., 1., 1.]),
 array([1., 1.]),
 array([1.   , 0.496])]